<h2> cv </h2>



In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Parameter
IMG_SIZE = 150
BATCH_SIZE = 32
EPOCHS = 15
BASE_DIR = "03_clean_desk"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    os.path.join(BASE_DIR, "train"),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

In [ ]:
# Model Definition
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
   train_generator,
   epochs=EPOCHS,
   steps_per_epoch = train_generator.samples // BATCH_SIZE
)

model.save(".h5")

In [ ]:
import tensorflow as tf
import pandas as pd
import os

test_dir = "03_clean_desk/test"
image_paths = [os.path.join(test_dir, fname) for fname in os.listdir(test_dir) if fname.lower().endswith(('.png','.jpg','.jpeg','.gif','.bmp'))]

IMG_SIZE = 150

def load_and_preprocess_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img /= 255.0
    return img

# Dataset 생성
test_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
test_dataset = test_dataset.map(load_and_preprocess_image).batch(BATCH_SIZE)

model = tf.keras.models.load_model(".h5")

predictions = model.predict(test_dataset)
predicted_labels = ["clean" if pred <0.5 else "not_clean" for pred in predictions.squeeze()]

result_df = pd.DataFrame({
    'file name' : [os.path.basename(fname) for fname in image_paths],
    'label' : predicted_labels
})


result_df.to_csv(".csv", index=False)

In [ ]:
import pandas as pd

result_path = ".csv"
loaded_df = pd.read_csv(result_path)

print(loaded_df.head())

In [ ]:
import os

test_dir = "03_clean_desk/test"
file_list = os.listdir(test_dir)

image_files = [f for f in file_list if f.lower().endswith(('.png', '.jpg', '.gif', '.bmp'))]

print(f"Number of image files in '{test_dir}' : {len(image_files)}")